## Cralwer dos sites de TI

In [ ]:
#Instalando o bs4
!pip install beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 22.3 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


### Imports

In [1]:
import math, sys, os

#Datas
import pandas as pd
import numpy as np

#Web
from bs4 import BeautifulSoup
import requests

#Plotters
import matplotlib as plt
import seaborn as sns

#Text
import re
import json

#handling dates
from datetime import datetime
import time

#remover dps
from pytictoc import TicToc

%load_ext autoreload
%autoreload 2

%time

ModuleNotFoundError: No module named 'bs4'

### Setando Header

In [ ]:
header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,
    'referer':'https://www.google.com/'
}

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
lista_url = df_a_consultar['loc'].to_list()

lista_vars_pichau = ['sku', 'potencia', 'name', 'stock_status', 'codigo_barra', 'is_openbox',
       'promotion_name', 'qty_available', 'qty_sold', 'expire_at',
       'url_image0', 'url_image1', 'value_promo', 'value_regular',
       'value_cash', 'dt_hr_captura']
       
df = pd.DataFrame(columns=lista_vars_pichau)

t = TicToc()

erros = []

c = 0
for i in lista_url:
    try:
        #t.tic()

        #try:
        c+=1
        #print(c)
        res = get_html_page(i)
        df_temp = extract_pichau(res)

        if df_temp.shape[0]>0:
            df = df.append(df_temp, ignore_index = True)
        #except Exception as e:
        #    traceback.print_exc()

        #t.toc()

        time.sleep(0.1515)
    except Exception as e:
        erros.append(f"{i}:{e}")
df

Houve um erro no processamento da url https://www.pichau.com.br/perifericos/mouse/mouse-gamer-dazz-fatality-3500dpi-usb-62171-0


,sku,potencia,name,stock_status,codigo_barra,is_openbox,promotion_name,qty_available,qty_sold,expire_at,url_image0,url_image1,value_promo,value_regular,value_cash,dt_hr_captura,url_image2,url_image3,url_image4,url_image5
0,B450MH,0,Placa Mae Biostar B450MH DDR4 Socket AM4 Chips...,OUT_OF_STOCK,4712960681736,0,NaN,NaN,NaN,NaN,https://media.pichau.com.br/media/catalog/prod...,https://media.pichau.com.br/media/catalog/prod...,636.25,636.25,559.90,2022-07-21 23:15:50.793409,https://media.pichau.com.br/media/catalog/prod...,https://media.pichau.com.br/media/catalog/prod...,NaN,NaN
1,SHQ1405BL/00,NaN,Fone de Ouvido Philips RunWild Esportivo com M...,OUT_OF_STOCK,6951613991800,0,NaN,NaN,NaN,NaN,https://media.pichau.com.br/media/catalog/prod...,https://media.pichau.com.br/media/catalog/prod...,90.79,90.79,79.90,2022-07-21 23:15:51.143384,https://media.pichau.com.br/media/catalog/prod...,NaN,NaN,NaN
2,GW800,NaN,"WebCam Redragon Hitman FHD 1080p USB, GW800",IN_STOCK,6950376778864,0,NaN,NaN,NaN,NaN,https://media.pichau.com.br/media/catalog/prod...,https://media.pichau.com.br/media/catalog/prod...,374.88,374.88,329.89,2022-07-21 23:15:51.495255,https://media.pichau.com.br/media/catalog/prod...,https://media.pichau.com.br/media/catalog/prod...,https://media.pichau.com.br/media/catalog/prod...,https://media.pichau.com.br/media/catalog/prod...
3,PCM-Mancer-Gamer-32229,NaN,"Computador Mancer Gamer Nefasta II, Intel i5-1...",IN_STOCK,NaN,0,PC Master Race,48,38,2022-07-31 23:50:00,https://media.pichau.com.br/media/catalog/prod...,https://media.pichau.com.br/media/catalog/prod...,6476.89,8107.01,5699.66,2022-07-21 23:15:51.803599,https://media.pichau.com.br/media/catalog/prod...,https://media.pichau.com.br/media/catalog/prod...,https://media.pichau.com.br/media/catalog/prod...,https://media.pichau.com.br/media/catalog/prod...
4,DUAL-GTX1050TI-O4G-V2,120,Placa de Video Asus Dual GeForce GTX 1050 Ti O...,OUT_OF_STOCK,4712900826159,0,NaN,NaN,NaN,NaN,https://media.pichau.com.br/media/catalog/prod...,https://media.pichau.com.br/media/catalog/prod...,860.35,860.35,757.11,2022-07-21 23:15:52.152225,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2337,TG-MP8-30-30-15-1R,NaN,"Almofada Termica Thermal Grizzly Minus PAD 8, ...",IN_STOCK,0753677507364,0,NaN,NaN,NaN,NaN,https://media.pichau.com.br/media/catalog/prod...,NaN,68.07,79.43,59.90,2022-07-21 23:44:47.811462,NaN,NaN,NaN,NaN
2338,CL-N004-PL20BL-A,NaN,"Base para Notebook Thermaltake Massive V20 17""...",OUT_OF_STOCK,4717964393418,0,NaN,NaN,NaN,NaN,https://media.pichau.com.br/media/catalog/prod...,https://media.pichau.com.br/media/catalog/prod...,113.52,113.52,99.90,2022-07-21 23:44:48.840634,NaN,NaN,NaN,NaN
2339,62466,NaN,"Mouse Gamer SteelSeries RIVAL 110 7200Dpi RGB,...",OUT_OF_STOCK,0813682024222,0,NaN,NaN,NaN,NaN,https://media.pichau.com.br/media/catalog/prod...,https://media.pichau.com.br/media/catalog/prod...,313.52,313.52,275.90,2022-07-21 23:44:49.835621,https://media.pichau.com.br/media/catalog/prod...,https://media.pichau.com.br/media/catalog/prod...,NaN,NaN
2340,TF12D432G3200HC16C01,NaN,Memoria Team Group T-Force Delta RGB 32GB (1x3...,OUT_OF_STOCK,0765441655546,0,NaN,NaN,NaN,NaN,https://media.pichau.com.br/media/catalog/prod...,https://media.pichau.com.br/media/catalog/prod...,1703.46,1703.46,1499.04,2022-07-21 23:44:50.218386,https://media.pichau.com.br/media/catalog/prod...,NaN,NaN,NaN


In [ ]:
_deepnote_run_altair(df, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":true},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"qty_available","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"COUNT(*)","type":"quantitative","sort":null,"aggregate":"count","scale":{"type":"linear","zero":true}},"color":{"field":"qty_sold","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [ ]:
#pesquisa
for ik in lista_url:
    if "md9" in ik:
        print(ik)

https://www.pichau.com.br/perifericos/cabos-e-adaptadores/cabo-md9-extensor-usb-macho-femea-1-80-metros
https://www.pichau.com.br/perifericos/cabos-e-adaptadores/cabo-hdmi-md9-1-8mt-1-4-filtro-tela-7615
https://www.pichau.com.br/perifericos/cabos-e-adaptadores/adaptador-md9-usb-para-fone-e-microfone-7674
https://www.pichau.com.br/perifericos/cabos-e-adaptadores/adaptador-md9-dvi-femea-para-hdmi-macho
https://www.pichau.com.br/perifericos/cabos-e-adaptadores/cabo-md9-p2-p2-2metros-7672
https://www.pichau.com.br/perifericos/cabos-e-adaptadores/cabo-extensor-usb-md9-3-metros-3-0-niquelado
https://www.pichau.com.br/perifericos/cabos-e-adaptadores/cabo-hdmi-md9-10-metros-7550
https://www.pichau.com.br/perifericos/cabos-e-adaptadores/cabo-extensor-usb-md9-10-metros-7483
https://www.pichau.com.br/perifericos/cabos-e-adaptadores/cabo-de-forca-md9-30-cm-hd-sata-preto
https://www.pichau.com.br/perifericos/cabos-e-adaptadores/adaptador-md9-hdmi-f-para-micro-hdmi-m-6634
https://www.pichau.com.br/p

In [ ]:
_deepnote_run_altair(df, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":true},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"name","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"qty_available","type":"nominal","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [ ]:
df_temp

,potencia,name,stock_status,codigo_barra,is_openbox,promotion_name,qty_available,qty_sold,expire_at,url_image0,url_image1,value_promo,value_regular,value_cash,dt_hr_captura
0,65,"Processador AMD Ryzen 5 5600, 6-Core, 12-Threa...",IN_STOCK,0730143314190,0,Ferias com o Pombo,83,264,2022-08-01 17:00:00,https://media.pichau.com.br/media/catalog/prod...,https://media.pichau.com.br/media/catalog/prod...,1269.32,1697.3,1117.0016,2022-07-21 22:12:07.605842


### Criando a função da Pichau

In [ ]:
def get_html_page(url : str, header=None) -> str:

    '''
    Funcao para o script HTML das paginas a serem monitoradas

    Retorno do metodo
    str

    Parametros

    url:str
    link do site a ser obtidos o script
    '''

    #Checa se existe o header
    if header is None  :
        header = {
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,
            'referer':'https://www.google.com/'
        }

    #Checa se a url está vazia antes de requisitar
    if len(url) == 0:
        print('URL vazia')
        return None

    r = requests.get(url, headers=header)
    
    return_code = r.status_code

    if not return_code in (200, 201, 202):
        print(f'Houve um erro no processamento da url {url}')
        return None
    
    return r.text


In [ ]:
res = get_html_page('https://www.pichau.com.br/processador-amd-ryzen-5-5600-6-core-12-threads-3-5ghz-4-4ghz-turbo-cache-35mb-am4-100-100000927box?utm_source=pcbuildwizard')

In [ ]:
def extract_pichau(html_script : str) -> pd.DataFrame:

    '''
    Funcao para buscar as informacoes do site da pichau e persisti-las

    Retorno do metodo
    pandas.DataFrame

    Parametros

    html_script:str
    script do site a ser obtidos as informações
    '''

    limite_galeria = 6

    df_retorno = pd.DataFrame()

    soup = BeautifulSoup(html_script, 'html.parser')

    all_data = soup.find('script', {'type':'application/json'})

    jsn = json.loads(all_data.string)

    #Usar esta lista de atributos neste nível: ['props']['pageProps']['pageData']['content']
    lista_attr_l0 = ['sku', 'potencia', 'name', 'stock_status', 'codigo_barra', 'is_openbox']

    for i in lista_attr_l0:
        df_retorno[i] = pd.Series(jsn['props']['pageProps']['pageData']['content'][i])

    #Usar esta lista de atributos neste nível: ['props']['pageProps']['pageData']['content']['mysales_promotion']
    lista_attr_l1 = ['promotion_name', 'qty_available', 'qty_sold', 'expire_at']

    if jsn['props']['pageProps']['pageData']['content']['mysales_promotion'] is not None:
        for i in lista_attr_l1:
            df_retorno[i] = pd.Series(jsn['props']['pageProps']['pageData']['content']['mysales_promotion'][i])

    galeria = jsn['props']['pageProps']['pageData']['content']['media_gallery']

    for img, counter in zip(galeria [:min(len(galeria), limite_galeria)], range(limite_galeria)):
        df_retorno["url_image" + str(counter)] = img['url']

    #df_retorno['currency_promo'] = pd.Series(jsn['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['final_price']['currency'])
    df_retorno['value_promo'] = pd.Series(jsn['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['final_price']['value'])

    #df_retorno['currency_regular'] = pd.Series(jsn['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['regular_price']['currency'])
    df_retorno['value_regular'] = pd.Series(jsn['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['regular_price']['value'])

    #df_retorno['currency_cash'] = df_retorno['currency_promo']
    df_retorno['value_cash'] = round(df_retorno['value_promo'].astype('float64') * 0.88, 2)

    df_retorno['dt_hr_captura'] = [datetime.now()]
    #print(df_retorno.shape)

    return df_retorno
    #data hora captura


In [ ]:
df = extract_pichau(res)

(1, 15)


In [ ]:
df.columns

Index(['potencia', 'name', 'stock_status', 'codigo_barra', 'is_openbox',
       'promotion_name', 'qty_available', 'qty_sold', 'expire_at',
       'url_image0', 'url_image1', 'value_promo', 'value_regular',
       'value_cash', 'dt_hr_captura'],
      dtype='object')

In [ ]:
#Usar esta lista de atributos neste nível: ['props']['pageProps']['pageData']['content']
lista_attr_l0 = ['potencia', 'name', 'stock_status', 'codigo_barra', 'is_openbox']

In [ ]:
for i in lista_attr_l0:
    print(jsn['props']['pageProps']['pageData']['content'][i])

65
Processador AMD Ryzen 5 5600, 6-Core, 12-Threads, 3.5GHz (4.4GHz Turbo), Cache 35MB, AM4, 100-100000927BOX
IN_STOCK
0730143314190
0


In [ ]:
#Usar esta lista de atributos neste nível: ['props']['pageProps']['pageData']['content']['mysales_promotion']
lista_attr_l1 = ['promotion_name', 'qty_available', 'qty_sold', 'expire_at']

In [ ]:
for i in lista_attr_l1:
    print(jsn['props']['pageProps']['pageData']['content']['mysales_promotion'][i])

Ferias com o Pombo
83
264
2022-08-01 17:00:00


In [ ]:
jsn['props']['pageProps']['pageData']['content']['mysales_promotion']

{'expire_at': '2022-08-01 17:00:00',
 'price_discount': 25.22,
 'price_promotional': 1269.32,
 'promotion_name': 'Ferias com o Pombo',
 'promotion_url': 'promocao/feriaspombo',
 'qty_available': 83,
 'qty_sold': 264,
 '__typename': 'MySalesPromotionProduct'}

In [ ]:
#moeda: ['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['final_price']['currency']
#value_promocional: ['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['final_price']['value']

#moeda: jsn['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['regular_price']['currency']
#value_regular: jsn['props']['pageProps']['pageData']['content']['price_range']['minimum_price']['regular_price']['value']


In [ ]:
galeria = jsn['props']['pageProps']['pageData']['content']['media_gallery']]
limit = 6
for i in galeria [:min(len(galeria), limit)]:
    print(i['url'])

https://media.pichau.com.br/media/catalog/product/cache/2f958555330323e505eba7ce930bdf27/1/0/100-100000927box_1.jpg
https://media.pichau.com.br/media/catalog/product/cache/2f958555330323e505eba7ce930bdf27/1/0/100-100000927box1_1.jpg


### Obtendo o sitemap da Pichau

In [ ]:
df_sitemap = pd.read_xml('https://www.pichau.com.br/media/sitemap.xml', headers=header)

TypeError: read_xml() got an unexpected keyword argument 'headers'

In [ ]:
r = requests.get('https://www.pichau.com.br/media/sitemap.xml', headers=header)
xml = r.text

In [ ]:
df_sitemap = pd.read_xml(xml)
df_sitemap[116:]

,loc,changefreq,priority,lastmod
116,https://www.pichau.com.br/hardware/placa-m-e/p...,daily,1.0,2021-08-12T14:02:51+00:00
117,https://www.pichau.com.br/hardware/placa-m-e/p...,daily,1.0,2021-08-12T14:02:17+00:00
118,https://www.pichau.com.br/hardware/placa-m-e/p...,daily,1.0,2021-08-12T14:01:55+00:00
119,https://www.pichau.com.br/hardware/placa-m-e/p...,daily,1.0,2021-08-12T14:01:39+00:00
120,https://www.pichau.com.br/hardware/placa-m-e/p...,daily,1.0,2021-08-12T14:01:34+00:00
...,...,...,...,...
20867,https://www.pichau.com.br/marca/zion,None,0.5,None
20868,https://www.pichau.com.br/marca/zogis,None,0.5,None
20869,https://www.pichau.com.br/marca/zotac,None,0.5,None
20870,https://www.pichau.com.br/marca/zowie,None,0.5,None


In [ ]:
df_a_consultar = df_sitemap[116:].sample(2390)

In [ ]:
jsn['props']['pageProps']['pageData']['content']

{'id': 31200,
 'sku': '100-100000927BOX',
 'url_key': 'processador-amd-ryzen-5-5600-6-core-12-threads-3-5ghz-4-4ghz-turbo-cache-35mb-am4-100-100000927box',
 'name': 'Processador AMD Ryzen 5 5600, 6-Core, 12-Threads, 3.5GHz (4.4GHz Turbo), Cache 35MB, AM4, 100-100000927BOX',
 'special_price': 1269.32,
 'socket': '539',
 'hide_from_search': 0,
 'is_openbox': 0,
 'openbox_state': None,
 'openbox_condition': None,
 'tipo_de_memoria': None,
 'caracteristicas': None,
 'slots_memoria': None,
 'marcas': 116,
 'marcas_info': {'name': 'AMD', '__typename': 'MarcasProduct'},
 'formato_placa': None,
 'plataforma': '7',
 'portas_sata': None,
 'slot_cooler_120': None,
 'slot_cooler_80': None,
 'slot_cooler_140': None,
 'slot_cooler_200': None,
 'coolerbox_included': 1,
 'potencia': '65',
 'quantidade_pacote': None,
 'alerta_monteseupc': None,
 'vgaintegrado': 153,
 'product_set_name': 'Processador',
 'categories': [{'name': 'Hardware', '__typename': 'CategoryTree'},
  {'name': 'Processadores', '__typ

In [ ]:
#teste url
r_teste = requests.get('https://www.kabum.com.br/produto/320798/processador-amd-ryzen-5-5600-cache-35mb-3-5ghz-4-4ghz-max-turbo-am4-sem-video-100-100000927box', headers=header)
r_teste.status_code

html_doc_teste = r_teste.text
soup_teste = BeautifulSoup(html_doc_teste, 'html.parser')
all_data_teste = soup_teste.find('script', {'type':'application/json'})
jsn_teste = json.loads(all_data_teste.string)
jsn_teste



{'props': {'pageProps': {'flagBlackFriday': False,
   'data': {'offerBannerImage': 'https://themes.kabum.com.br/conteudo/oferta/108/banner/julho-gamer_descricao.jpg',
    'productCatalog': '{"code":320798,"productSpecie":0,"name":"Processador AMD Ryzen 5 5600, Cache 35MB, 3.5GHz (4.4GHz Max Turbo), AM4, Sem Vídeo - 100-100000927BOX","description":"<h2>Processador AMD Ryzen 5 5600</h2>\\n<p>&nbsp;</p>\\n<p>Quando voc&ecirc; tem a arquitetura de processador&nbsp;de desktop&nbsp;mais avan&ccedil;ada do mundo2&nbsp;para jogadores e criadores de conte&uacute;do, as possibilidades s&atilde;o infinitas. Esteja voc&ecirc; jogando os jogos mais recentes, projetando o pr&oacute;ximo pr&eacute;dio ou processando dados, voc&ecirc; precisa de um processador poderoso que possa lidar com tudo - e muito mais. Sem d&uacute;vida, os processadores AMD Ryzen &trade; s&eacute;rie 5000 definem o padr&atilde;o para jogadores e artistas.</p>\\n<p>&nbsp;</p>\\n<h2>A ferramenta certa para o trabalho</h2>\\n<div

In [ ]:
aaaaa

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b8b98786-19ce-460b-98f4-40a20facf290' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>